In [30]:
from pyspark.sql import SparkSession
import requests
import pandas as pd
from pyspark.sql.functions import max, count

In [31]:
spark = SparkSession.builder \
    .appName("Whisky Auction Data Processing Lab") \
    .getOrCreate()

In [32]:
# Отримання даних з Whisky API
api_url = "https://whiskyhunter.net/api/auctions_data/"
response = requests.get(api_url)
auctions_data = response.json()

In [33]:
# Перетворио дані у DataFrame
auctions = [{
    'auction_date': auction.get('dt'),
    'auction_name': auction.get('auction_name'),
    'auction_slug': auction.get('auction_slug'),
    'winning_bid_max': auction.get('winning_bid_max', 0),
    'winning_bid_min': auction.get('winning_bid_min', 0),
    'winning_bid_mean': auction.get('winning_bid_mean', 0),
    'auction_trading_volume': auction.get('auction_trading_volume', 0),
    'auction_lots_count': auction.get('auction_lots_count', 0)
} for auction in auctions_data]

In [34]:
df_auctions = pd.DataFrame(auctions)
auctions_df = spark.createDataFrame(df_auctions)
auctions_df.show(truncate=False)

+------------+---------------------------+--------------------------+---------------+---------------+----------------+----------------------+------------------+
|auction_date|auction_name               |auction_slug              |winning_bid_max|winning_bid_min|winning_bid_mean|auction_trading_volume|auction_lots_count|
+------------+---------------------------+--------------------------+---------------+---------------+----------------+----------------------+------------------+
|2024-09-01  |Australian Whisky Auctions |australian-whisky-auctions|8014.0         |4.7            |189.47          |202164.7              |1067              |
|2024-09-01  |Bonhams Whisky Auction     |bonhams                   |7000.0         |190.0          |964.31          |62680.0               |65                |
|2024-09-01  |Catawiki Whisky Auction    |catawiki                  |12637.2        |0.8            |174.67          |950225.3              |5440              |
|2024-09-01  |Celtic Whiskey Aucti

In [35]:
# кількість лотів більше 5000
filtered_auctions = auctions_df.filter(auctions_df.auction_lots_count > 5000)
filtered_auctions.show(truncate=False)

+------------+-----------------------+--------------------+---------------+---------------+----------------+----------------------+------------------+
|auction_date|auction_name           |auction_slug        |winning_bid_max|winning_bid_min|winning_bid_mean|auction_trading_volume|auction_lots_count|
+------------+-----------------------+--------------------+---------------+---------------+----------------+----------------------+------------------+
|2024-09-01  |Catawiki Whisky Auction|catawiki            |12637.2        |0.8            |174.67          |950225.3              |5440              |
|2024-09-01  |Unicorn Auctions       |unicorn-auctions    |27736.5        |3.8            |243.4           |2866065.9             |11775             |
|2024-08-01  |Scotch Whisky Auctions |scotchwhiskyauctions|48000.0        |15.0           |204.1           |2299235.0             |11265             |
|2024-08-01  |Unicorn Auctions       |unicorn-auctions    |28947.2        |3.9            |239

In [36]:
max_bid_per_auction = auctions_df.groupBy("auction_name", "auction_date").agg(max("winning_bid_max").alias("max_bid"))
max_bid_per_auction.show(truncate=False)

+------------------------+------------+--------+
|auction_name            |auction_date|max_bid |
+------------------------+------------+--------+
|Highland Whisky Auctions|2024-04-01  |5100.0  |
|Just Whisky Auctions    |2024-02-01  |7500.0  |
|The Grand Whisky Auction|2024-08-01  |4150.0  |
|Whisky.Auction          |2024-05-01  |6400.0  |
|Bonhams Whisky Auction  |2024-05-01  |41699.1 |
|European Whisky Auctions|2024-04-01  |1651.4  |
|Celtic Whiskey Auction  |2024-01-01  |2943.8  |
|Unicorn Auctions        |2023-11-01  |20027.1 |
|Prestige Whisky Auction |2024-01-01  |9000.0  |
|Whiskey Bidders         |2023-11-01  |6313.3  |
|Whisky Auctioneer       |2024-06-01  |50000.0 |
|Unicorn Auctions        |2024-09-01  |27736.5 |
|WhiskyAuction.com       |2024-04-01  |9331.2  |
|Whisky Hammer           |2023-12-01  |110000.0|
|Irish Whiskey Auctions  |2024-09-01  |3971.5  |
|McTear’s Whisky Auction |2024-05-01  |3400.0  |
|Catawiki Whisky Auction |2024-03-01  |6875.6  |
|The Grand Whisky Au

In [ ]:
spark.stop()